In [20]:
import pandas as pd
import pybamm
import pybop

In [33]:
parameter_set = pybamm.ParameterValues("ECM_Example")
parameter_set.update(
    {
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        # "Current function [A]": 5,
        # "Initial SoC": 0.5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)
# Optional arguments - only needed for two RC pairs
parameter_set.update(
    {
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

In [34]:
model = pybop.empirical.Thevenin(
    parameter_set=parameter_set,
    options={"number of rc elements": 2},
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)

file_loc = r"../hppc_lut/G1/battery_G1_cycle_2_pulse_0_lut.csv"
df = pd.read_csv(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy() * -1, 
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)

In [51]:
r_guess = 0.005
parameters = pybop.Parameters(
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[1, 1500],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[0, 25000],
    ),
)

In [52]:
model.build(
    initial_state={"Initial open-circuit voltage [V]": df["Voltage"].to_numpy()[0]}
)
problem = pybop.FittingProblem(
    model,
    parameters,
    dataset,
)

cost = pybop.SumSquaredError(problem)

In [53]:
optim = pybop.PSO(
    cost,
    sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
    max_unchanged_iterations=30,
    max_iterations=100,
)
results = optim.run()

Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [5.79437868e-03 3.89119403e-03 5.21867552e-03 3.46774099e+02
 2.97816063e+03]
  Optimised parameters: [1.86214942e-02 1.87859977e-02 8.35615347e-03 2.56204851e+01
 2.25918953e+03]
  Total-order sensitivities:
  Diagonal Fisher Information entries: None
  Final cost: 0.6506986937366275
  Optimisation time: 77.27011799812317 seconds
  Number of iterations: 48
  Number of evaluations: 282
  SciPy result available: No
  PyBaMM Solution available: Yes


In [56]:
pybop.plot.quick(problem, problem_inputs=results.x, title="Optimised Comparison");

In [57]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

In [2]:
import pandas as pd
import pybamm
import pybop
import glob

def parameterize_pulse(file_loc):
    # Read the pulse data
    df = pd.read_csv(file_loc, index_col=None, na_values=["NA"])
    df = df.drop_duplicates(subset=["Time"], keep="first")
    
    # Create dataset for pybop
    dataset = pybop.Dataset({
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy() * -1,  # adjust sign if needed
        "Voltage [V]": df["Voltage"].to_numpy(),
    })
    
    # Set up parameter set and update as needed
    parameter_set = pybamm.ParameterValues("ECM_Example")
    parameter_set.update(
        {
            "Cell capacity [A.h]": 5,
            "Nominal cell capacity [A.h]": 5,
            "Element-1 initial overpotential [V]": 0,
            "Upper voltage cut-off [V]": 4.2,
            "Lower voltage cut-off [V]": 3.0,
            "R0 [Ohm]": 1e-3,
            "R1 [Ohm]": 2e-4,
            "C1 [F]": 1e4,
            "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values["Open-circuit voltage [V]"],
        }
    )
    parameter_set.update(
        {
            "R2 [Ohm]": 0.0003,
            "C2 [F]": 40000,
            "Element-2 initial overpotential [V]": 0,
        },
        check_already_exists=False,
    )
    
    # Build the model with two RC pairs
    model = pybop.empirical.Thevenin(
        parameter_set=parameter_set,
        options={"number of rc elements": 2},
        solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
    )
    
    model.build(initial_state={"Initial open-circuit voltage [V]": df["Voltage"].to_numpy()[0]})
    
    # Define parameters with priors and bounds
    r_guess = 0.005
    parameters = pybop.Parameters(
        pybop.Parameter("R0 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("R1 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("R2 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("C1 [F]", prior=pybop.Gaussian(500, 100), bounds=[1, 1500]),
        pybop.Parameter("C2 [F]", prior=pybop.Gaussian(2000, 500), bounds=[0, 25000]),
    )
    
    problem = pybop.FittingProblem(model, parameters, dataset)
    
    cost = pybop.SumSquaredError(problem)
    optim = pybop.PSO(cost, sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
                       max_unchanged_iterations=30, max_iterations=100)
    
    results = optim.run()

    # Extract optimized parameters using their array indices
    r0_opt = results.x[0]  # R0
    r1_opt = results.x[1]  # R1
    r2_opt = results.x[2]  # R2
    c1_opt = results.x[3]  # C1
    c2_opt = results.x[4]  # C2
    
    return r0_opt, r1_opt, r2_opt, c1_opt, c2_opt

# Define constant operating conditions
constant_temperature = 25   # [degC]
constant_soc = 0.5          # SoC as a fraction

# Prepare empty lists to accumulate LUT rows
R0_lut = []
R1_lut = []
R2_lut = []
C1_lut = []
C2_lut = []

# Loop over pulse files (adjust the path as needed)
pulse_files = sorted(glob.glob(r"../hppc_lut/G1/battery_G1_cycle_2_pulse_*_lut.csv"))
for file_loc in pulse_files:
    try:
        r0_opt, r1_opt, r2_opt, c1_opt, c2_opt = parameterize_pulse(file_loc)
    except Exception as e:
        print(f"Error processing {file_loc}: {e}")
        continue
    
    # Use a representative current value from the file (here, the first current value)
    df_temp = pd.read_csv(file_loc)
    representative_current = df_temp["Current"].to_numpy()[0]
    
    R0_lut.append({
        "Temperature [degC]": constant_temperature,
        "Current [A]": representative_current,
        "SoC": constant_soc,
        "R0 [Ohm]": r0_opt,
    })
    R1_lut.append({
        "Temperature [degC]": constant_temperature,
        "Current [A]": representative_current,
        "SoC": constant_soc,
        "R1 [Ohm]": r1_opt,
    })
    R2_lut.append({
        "Temperature [degC]": constant_temperature,
        "Current [A]": representative_current,
        "SoC": constant_soc,
        "R2 [Ohm]": r2_opt,
    })
    C1_lut.append({
        "Temperature [degC]": constant_temperature,
        "Current [A]": representative_current,
        "SoC": constant_soc,
        "C1 [F]": c1_opt,
    })
    C2_lut.append({
        "Temperature [degC]": constant_temperature,
        "Current [A]": representative_current,
        "SoC": constant_soc,
        "C2 [F]": c2_opt,
    })

# Convert lists to DataFrames and save as CSV files
R0_df = pd.DataFrame(R0_lut)
R1_df = pd.DataFrame(R1_lut)
R2_df = pd.DataFrame(R2_lut)
C1_df = pd.DataFrame(C1_lut)
C2_df = pd.DataFrame(C2_lut)

R0_df.to_csv("R0_lut.csv", index=False)
R1_df.to_csv("R1_lut.csv", index=False)
R2_df.to_csv("R2_lut.csv", index=False)
C1_df.to_csv("


Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [5.11653499e-03 5.36123207e-03 5.07469119e-03 3.21476068e+02
 2.59326200e+03]
  Optimised parameters: [3.42980451e-02 6.03492440e-03 5.41612155e-03 9.81190930e+02
 2.98268866e+03]
  Total-order sensitivities:
  Diagonal Fisher Information entries: None
  Final cost: 0.646570920665213
  Optimisation time: 84.99266290664673 seconds
  Number of iterations: 69
  Number of evaluations: 453
  SciPy result available: No
  PyBaMM Solution available: Yes


c:\Users\markv\AppData\Local\Programs\Python\Python311\Lib\site-packages\pints\_optimisers\_pso.py:264: UserWarning: All PSO particles are outside the boundaries.
  warnings.warn('All PSO particles are outside the boundaries.')



----------------------------------------
Unexpected termination.
Current score: (0.08555247590245849, 0.08555247590245849)
Current position:
 3.37156172737508447e-02
 4.79058595454874385e-03
 7.86104390403434897e-03
 7.68745060638583254e+02
 4.56622076023672525e+03
----------------------------------------


KeyboardInterrupt: 